# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
weather_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Opunake,87,NZ,1586799408,84,-39.46,173.86,48.99,10.00
1,Pangnirtung,90,CA,1586798953,92,66.15,-65.71,17.60,13.87
2,Punta Arenas,75,CL,1586799188,62,-53.15,-70.92,51.80,31.09
3,Albany,90,US,1586799210,82,42.60,-73.97,63.00,20.80
4,Bredasdorp,2,ZA,1586799413,59,-34.53,20.04,60.80,6.02


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lng"]]

Weight = weather_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [24]:
hotel_df = weather_df.loc[(weather_df['Max Temp'] > 70) & (weather_df['Max Temp'] < 80) & 
                          (weather_df['Wind Speed'] < 10) & (weather_df['Cloudiness'] == 0)]

hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
133,Ures,0,MX,1586799572,34,29.43,-110.40,75.99,7.00
152,Rāzole,0,IN,1586799596,81,16.48,81.83,79.14,3.85
244,Kununurra,0,AU,1586799712,94,-15.77,128.73,73.40,3.24
355,Hīt,0,IQ,1586799845,22,33.64,42.83,70.97,3.33
358,Sakakah,0,SA,1586799561,13,29.97,40.21,75.20,4.70
388,Amboasary,0,MG,1586799885,80,-25.03,46.38,75.58,5.84
457,Hurghada,0,EG,1586799969,40,27.26,33.81,73.40,9.17
537,San Juan,0,AR,1586799847,27,-31.54,-68.54,72.09,6.62
547,Umluj,0,SA,1586800074,56,25.02,37.27,73.49,8.30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"key": g_key, "radius": 5000, "keyword": "hotels"}

In [27]:
for index, row in hotel_df.iterrows():
    
    params["location"] = (f"{row['Lat']},{row['Lng']}")
    
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        print(f" Hotel found in the city of {row['City']}")
              
    except(KeyError, IndexError):
              
        print(f" Sorry, could not find a hotel near {row['City']}")

 Hotel found in the city of Ures
 Hotel found in the city of Rāzole
 Hotel found in the city of Kununurra
 Sorry, could not find a hotel near Hīt
 Hotel found in the city of Sakakah
 Hotel found in the city of Amboasary
 Hotel found in the city of Hurghada
 Hotel found in the city of San Juan
 Hotel found in the city of Umluj


In [35]:
# Dropping observation for which there is no hotel found

hotel_df_complete = hotel_df.loc[hotel_df['Hotel Name'].notna()]

hotel_df_complete

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
133,Ures,0,MX,1586799572,34,29.43,-110.40,75.99,7.00,Hacienda del Labrador
152,Rāzole,0,IN,1586799596,81,16.48,81.83,79.14,3.85,Hotel Somisetty Landmark Konaseema
244,Kununurra,0,AU,1586799712,94,-15.77,128.73,73.40,3.24,Freshwater East Kimberley Apartments | Kununur...
358,Sakakah,0,SA,1586799561,13,29.97,40.21,75.20,4.70,Raoum Inn Hotel
388,Amboasary,0,MG,1586799885,80,-25.03,46.38,75.58,5.84,DISCOVERY AMBOASARY
457,Hurghada,0,EG,1586799969,40,27.26,33.81,73.40,9.17,MERAKI Resort
537,San Juan,0,AR,1586799847,27,-31.54,-68.54,72.09,6.62,Hotel Viñas del Sol
547,Umluj,0,SA,1586800074,56,25.02,37.27,73.49,8.30,عروس الشاطئ للوحدات السكنية المفروشة


In [37]:
narrowed_city_df = hotel_df_complete.loc[:,['City', 'Country', 'Hotel Name']]

In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]

locations = hotel_df_complete[["Lat", "Lng"]]

fig = gmaps.figure()

markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)

fig.add_layer(markers)

In [41]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

fig.add_layer(heat_layer)

fig.add_layer(markers)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))